In [1]:
import cairo_jupyter
print(cairo_jupyter)

<module 'cairo_jupyter' from '/home/stu/.local/lib/python3.8/site-packages/cairo_jupyter/__init__.py'>


In [2]:
%reload_ext cairo_jupyter

In [3]:
#!pip install ipdb ipython

In [ ]:
import asyncio
import enum
import re
import sys
import traceback

from aiohttp import web

import cairo

from contextlib import contextmanager
from functools import partialmethod
from io import BytesIO
from pathlib import Path

from IPython.display import display, clear_output

FORMAT_PS = 'ps'
FORMAT_PDF = 'pdf'
FORMAT_PNG = 'png'
FORMAT_SVG = 'svg'

VALID_FORMATS = [FORMAT_PNG, FORMAT_SVG]

ERROR_UNSUPPORTED_FORMAT = "Format must be FORMAT_PDF, FORMAT_PNG, FORMAT_PS or FORMAT_SVG"



class RunObserver:
    """
    RunObserver is a base class that allowing the user to asynchronously block until source has
    executed successfully or raised an exception.
    """
    def __init__(self):
        self._event = asyncio.Event()

    async def wait_until_after_run(self):
        """
        Wait for source to run an iteration or throw exception.
        """
        await self._event.wait()
    
    def run_was_successful(self, source):
        """
        Called after a successful run of the source.
        Unblocks callers to wait_until_after_run.   
        
        Subclasses may save information to be retrieved by wait_until_after_run.
        """
        self._event.set()
        
    def run_raised_exception(self, source, exception):
        """
        Called if attempting to run the source raised an exception.
        Unblocks callers to wait_until_after_run.   
        
        Subclasses may save information to be retrieved by wait_until_after_run.
        """
        self._event.set()

    
class SourceChanged(RunObserver):
    class Status(enum.Enum):
        WAITING = 1
        NOT_UPDATED = 2
        UPDATED_SUCCESSFULLY = 3
        UPDATED_BY_ANOTHER = 4

    def __init__(self, source):
        RunObserver.__init__(self)
        
        self.success = None
        self.exception = None
        
        self.source = source
        self.status = SourceChanged.Status.WAITING
        self.updated_by = None

    def source_updated(self, updated_by):
        """
        Called when source was updated, by this SourceChanged or
        another.
        
        If updated_by was this SourceChanged set status to UPDATED_SUCCESSFULLY and continue waiting.
        
        If updated_by is None then set status to NOT_UPDATED and finish waiting.
        If updated_by is not self set status to UPDATED_BY_ANOTHER and finish waiting.
        """
        self.updated_by = updated_by
        if updated_by == self:
            self.status = SourceChanged.Status.UPDATED_SUCCESSFULLY
            # Don't stop waiting here
            return True
        
        if updated_by is None:
            self.status = SourceChanged.Status.NOT_UPDATED
        elif updated_by != self:
            self.status = SourceChanged.Status.UPDATED_BY_ANOTHER
        self._event.set()
    
    def run_was_successful(self, source):
        """
        Called after a successful run of the source.
        Unblocks callers to wait_until_after_run.   
        
        Saves the successfully run source to self.source.
        """
        self.source = source
        self.exception = None
        self._event.set()
        
    def run_raised_exception(self, source, exception):
        """
        Called if attempting to run the source raised an exception.
        Unblocks callers to wait_until_after_run.   
        
        Save the failed source and the exception then allow wait_until_after_run
        to stop waiting.
        """
        self.source = source
        self.exception = exception
        self._event.set()
        
    async def wait_until_after_run(self):
        """
        Wait for source to run an iteration or throw exception.
        
        If run_raised_exception was called then its exception is raised.
        """
        await super().wait_until_after_run()
        if self.exception:
            raise self.exception

      
        
# Shoebot Grammar        
class Grob:
    def __init__(self, ctx):
        self._ctx = ctx

#  nodebox does:
#     def draw(self):
#         self._ctx.append(self)

        
class Channels(enum.Enum):
    Alpha = 0
    Red = 1
    Green = 2
    Blue = 3
    Intensity = 4

BGRA = (
    Channels.Blue,
    Channels.Green,
    Channels.Red,
    Channels.Alpha
)
RGB = (
    Channels.Red,
    Channels.Green,
    Channels.Blue,
)
RGBA = (
    Channels.Red,
    Channels.Green,
    Channels.Blue,
    Channels.Alpha
)

GRAYSCALE = (Channels.Intensity,)
ALPHA = Channels.Alpha
GRAYSCALE_ALPHA = (Channels.Intensity, Channels.Alpha)
TRANSPARENT = ()

class Color:
    def __init__(self, *args, **kwargs):
        channels = kwargs.get('channels')
        if channels is None:
            if len(args) == 0:
                channels = TRANSPARENT
            elif len(args) == 1:
                if args[0] is None:
                    channels = TRANSPARENT
                elif isinstance(args[0], Color):
                    color = args[0]
                    channels, args = tuple(color.data.items())
                else:
                    channels = GRAYSCALE
            elif len(args) == 2:
                 channels = GRAYSCALE_ALPHA
            elif len(args) == 3:
                channels = RGB
            elif len(args) == 4:
                channels = RGBA
            else:
                raise ValueError(f"Unknown color arrangement: {args}")

        if len(args) != len(channels):
            raise ValueError(f"len(args) != channel(args): args={args}, channels={channels}")
            
        # Build dict of {channel: color} from tuples of (channel, color)
        self.data = dict(zip(channels, args))
    
    @property
    def is_transparent(self):
        if not self.data:
            return True
        
        return self.alpha == 0.0
    
    @property
    def alpha(self):
        if not self.data:
            return 0.0
        
        return self.data.get(Channels.Alpha, 1.0)

    def calculate_intensity(self):
        """Calculate intensity from colour channels.
        
        If there are no colour channels, returns zero.
        """
        intensity_channels = 0
        intensity = 0
        
        for channel, value in self.data.items():
            if channel in (Channels.Alpha, Channels.Intensity):
                continue
            intensity_channels += 1
            intensity += value
            
        if intensity == 0:
            return 0.0
        
        return intensity / intensity_channels        
    
    @property
    def intensity(self):
        """Intensity - for greyscale images this is the intensity channel (grey)
        for colour images this is calculated from the colour channels.
        """
        intensity = self.data.get(Channels.Intensity)
        if intensity is not None:
            return intensity
        
        return self.data.get(Channels.Intensity, self.calculate_intensity())
    
    @property
    def r(self):
        value = self.data.get(Channels.Red)
        if value is not None:
            return value

        if self.is_transparent:
            return 0.0

        return self.data[Channels.Intensity]
    
    @property
    def g(self):
        value = self.data.get(Channels.Green)
        if value is not None:
            return value

        if self.is_transparent:
            return 0.0

        return self.data[Channels.Intensity]
    
    @property
    def b(self):
        value = self.data.get(Channels.Blue)
        if value is not None:
            return value

        if self.is_transparent:
            return 0.0

        return self.data[Channels.Intensity]
    
    @property
    def rgba(self):
        return (
            self.r,
            self.g,
            self.b,
            self.alpha,
        )
    
    @property
    def rgb(self):
        return (
            self.r,
            self.g,
            self.b,
        )
    
    def __str__(self):
        # TODO - make output consistent - you can't acually pass data as a param
        return f"<Color data={self.data}>"


    
class ColorMixin:
    """
    Color state:
        fill: (r, g, b, a)
        stroke: (r, g, b, a)
        blendmode: TODO enum, like BlendMode.THROUGH
    """
    # TODO - is this really a color, a different name
    # pen, drawable, paintable Colorable ???
    def __init__(self, fill=None, stroke=None, blendmode=None, **kwargs):
        if isinstance(fill, Color):
            self._fill = fill
        else:
            self._fill = Color(*fill)

        if isinstance(stroke, Color):
            self._stroke = stroke
        else:
            self._stroke = Color(*stroke)

        self._blendmode = blendmode

# TODO have state internal ?  ColorState, TranslationState etc ?
# class PathState(ColorState, TranslationState)
# self._state = PathState(....)
# self._state.fill = (1., 1., .5)
# self._state.blendmode = BlendMode.xor

MOVETO = "moveto"
RMOVETO = "rmoveto"
LINETO = "lineto"
RLINETO = "rlineto"
CURVETO = "curveto"
RCURVETO = "rcurveto"
ARC = "arc"
ELLIPSE = "ellipse"
CLOSE = "close"


class Point:
    def __init__(self, *args):
        if len(args) == 2:
            self.x, self.y = args
        elif len(args) == 1:
            self.x, self.y = args[0]
        elif len(args) == 0:
            self.x = self.y = 0.0
        else:
            raise ValueError("Point accepts 0, 1, or 2 arguments.")


class PathElement:
    # Taken from Nodebox and extended.

    def __init__(self, cmd=None, pts=None):
        self.cmd = cmd
        if cmd in (MOVETO, RMOVETO):
            assert len(pts) == 1
            self.x, self.y = pts[0]
            self.ctrl1 = Point(pts[0])
            self.ctrl2 = Point(pts[0])
        elif cmd in (LINETO, RLINETO):
            assert len(pts) == 1
            self.x, self.y = pts[0]
            self.ctrl1 = Point(pts[0])
            self.ctrl2 = Point(pts[0])
        elif cmd in (CURVETO, RCURVETO):
            assert len(pts) == 3
            self.ctrl1 = Point(pts[0])
            self.ctrl2 = Point(pts[1])
            self.x, self.y = pts[2]
        elif cmd == CLOSE:
            assert pts is None or len(pts) == 0
            self.x = self.y = 0.0
            self.ctrl1 = Point(0.0, 0.0)
            self.ctrl2 = Point(0.0, 0.0)
        else:
            self.x = self.y = 0.0
            self.ctrl1 = Point()
            self.ctrl2 = Point()
            
    def as_cairo(self, ctx):
        """
        Run command corresponding to this PathElement on a cairo Context.
        """
        if self.cmd == MOVETO:
            ctx.move_to(self.x, self.y)
        elif self.cmd == RMOVETO:
            # actually dx, dy
            ctx.rel_move_to(self.x, self.y)
        elif self.cmd == LINETO:
            ctx.line_to(self.x, self.y)
        elif self.cmd == RLINETO:
            # actually dx, dy
            ctx.rel_line_to(self.x, self.y)
        elif self.cmd == CURVETO:
            ctx.curve_to(self.x, self.y,
                         self.ctrl1.x, self.ctrl1.y,
                         self.ctrl2.x, self.ctrl2.y)
        elif self.cmd == RCURVETO:
            # actually dx, dy
            ctx.rel_curve_to(self.x, self.y,
                             self.ctrl1.x, self.ctrl1.y,
                             self.ctrl2.x, self.ctrl2.y)
        elif self.cmd == CLOSE:
            ctx.close_path()

    def __repr__(self):
        if self.cmd == MOVETO:
            return f"<PathElement(MOVETO, (({self.x:.3f}, {self.y:.3f}),))>"
        elif self.cmd == RMOVETO:
            return f"<PathElement(RMOVETO, (({self.x:.3f}, {self.y:.3f}),))>"
        elif self.cmd == LINETO:
            return f"<PathElement(LINETO, (({self.x:.3f}, {self.y:.3f}),))>"
        elif self.cmd == RLINETO:
            return f"<PathElement(RLINETO, (({self.x:.3f}, {self.y:.3f}),))>"
        elif self.cmd == CURVETO:
            return f"<PathElement(CURVETO, (({self.ctrl1.x:.3f}, {self.ctrl1.y:.3f}), ({self.ctrl2.x:.3f}, {self.ctrl2.y}), ({self.x:.3f}, {self.y:.3f}))>"
        elif self.cmd == RCURVETO:
            return f"<PathElement(RCURVETO, (({self.ctrl1.x:.3f}, {self.ctrl1.y:.3f}), ({self.ctrl2.x:.3f}, {self.ctrl2.y}), ({self.x:.3f}, {self.y:.3f}))>"
        elif self.cmd == CLOSE:
            return "<PathElement(CLOSE)>"
            
    def __eq__(self, other):
        if other is None:
            return False
        if self.cmd != other.cmd:
            return False
        return self.x == other.x and self.y == other.y \
            and self.ctrl1 == other.ctrl1 and self.ctrl2 == other.ctrl2
        
    def __ne__(self, other):
        return not self.__eq__(other)


class Path(Grob, ColorMixin):
    f"""
    Path containing Beziers and straight lines.
    
    {ColorMixin.__doc__}
    """
    def __init__(self, ctx, **kwargs):
        Grob.__init__(self, ctx)
        ColorMixin.__init__(self, ctx._fill, ctx._stroke)
        self._elements = []
        self.closed = False
        
    def append(self, element):
        self._elements.append(element)
        return self
    
    def moveto(self, x, y):
        self.append(PathElement(MOVETO, ((x, y),)))

    def lineto(self, x, y):
        self.append(PathElement(LINETO, ((x, y),)))

    def curveto(self, x1, y1, x2, y2, x3, y3):
        self.append(
            PathElement(CURVETO, (x1, y1), (x2, y2), (x3, y3))
        )

    def closepath(self):
        if self._elements:
            element = self._elements[0]
            self.append(PathElement(CLOSE))
            self.closed = True

#     def ellipse(self, x, y, w, h, ellipsemode=CORNER):
#         # convert values if ellipsemode is not CORNER
#         if ellipsemode == CENTER:
#             x = x - (w / 2)
#             y = y - (h / 2)
#         elif ellipsemode == CORNERS:
#             w = w - x
#             h = h - y
#         self._append_element(
#             self._canvas.ellipse_closure(x, y, w, h), (ELLIPSE, x, y, w, h)
#         )
#         self.closed = True

    def rellineto(self, x, y):
        self.append(PathElement(RLINETO, ((x, y),),))

    # High level API
    def line(self, x1, y1, x2, y2):
        self.moveto(x1, y1)
        self.lineto(x2, y2)

    def rect(self, x, y, w, h, roundness=0.0): #, rectmode=CORNER):
        # TODO - cornermode.
        # convert values if rectmode is not CORNER
        #         if rectmode == CENTER:
        #             x = x - (w / 2.)
        #             y = y - (h / 2.)
        #         elif rectmode == CORNERS:
        #             w = w - x
        #             h = h - y
        if not roundness:
            self.moveto(x, y)
            self.rellineto(w, 0)
            self.rellineto(0, h)
            self.rellineto(-w, 0)
            self.closepath()
        else:
            # rounded rect.
            curve = min(w * roundness, h * roundness)
            self.moveto(x, y + curve)
            self.curveto(x, y, x, y, x + curve, y)
            self.lineto(x + w - curve, y)
            self.curveto(x + w, y, x + w, y, x + w, y + curve)
            self.lineto(x + w, y + h - curve)
            self.curveto(x + w, y + h, x + w, y + h, x + w - curve, y + h)
            self.lineto(x + curve, y + h)
            self.curveto(x, y + h, x, y + h, x, y + h - curve)
            self.closepath()
    
    def draw(self):
        self._ctx._canvas.draw_path(self,
                                    self._fill,
                                    self._stroke)
        return self

    def as_cairo(self, ctx):
        """
        Run path commands on a cairo Context.
        """
        for element in self._elements:
            element.as_cairo(ctx)
        return self
    
    def __repr__(self):
        return f"<Path {self._elements}>"

_Path = Path

class CanvasBase:
    ## TODO - this isn't used, trying to work out if the API is right by prototyping this.
    def new_page(self, context):
        raise NotImplemented()
        
    def draw_path(seslf, path, fill, stroke):
        raise NotImplemented()
        
    def save(self, f, format=None, *args, **kwargs):
        # TODO: naming - f could be dest ?
        raise NotImplemented()
        
    @contextmanager
    def recording_surface(self, extents=None):
        # TODO: naming - This definitely shows the name is too cairo
        raise NotImplemented()
        
    def display(self, format=None, *args, **kwargs):
        # TODO: should this be here... (maybe jupyter support should be seperate?)
        #       For:  everything should have jupyter support
        #       Against:  No it shouldn't
        #       Alternative:  Have different put targets, think about how it would work
        #                     in a DAG/Node based world.
        raise NotImplemented()
        
        
        

class Canvas:
    """
    Uses a RecordingSurface to keep track of the draw state on the current frame/page,
    and allowing it to be copied to various outputs as needed.
    """
    def __init__(self, extents: (float, float, float, float) = None, resizable=None, **kwargs):
        """
        :param extents: Extents of recording surface, or None for unbounded.
        :param resizable: If True, the surface will be resizable.
        """
        self.resizable = resizable or extents is None
        self.surface = self.create_recording_surface(extents)
        self.position = (0, 0)
        self.extents = extents

    def create_recording_surface(self, extents=None):
        return cairo.RecordingSurface(cairo.CONTENT_COLOR_ALPHA, extents)

    def resize(self, dimensions, clear=True):
        if not self.resizable:
            return False

        extents = (0, 0, dimensions[0], dimensions[1])
        if self.extents == extents:
            return True

        self.extents = extents
        new_surface = self.create_recording_surface(extents)

        if not clear:
            # Copy the old surface to the new one.
            with new_surface.get_context() as ctx:
                ctx.set_source_surface(self.surface, 0, 0)
                ctx.paint()

        self.surface = new_surface
        return True

    def translate(self, x, y):
        # TODO - added to get Shoebot.bot working
        #        think about how position should really work
        self.position += (x, y)
        
    @contextmanager
    def new_page(self, context):
        """:param context: Context object is passed to retrieve the background after execution.
        """
        ctx = cairo.Context(self.surface)
        
        ctx.push_group()
        try:
            yield
        except:
            # Don't leak resources if something went wrong.
            ctx.pop_group()
            raise
            
        background = context._background
        if background is not None:
            ctx.set_source_rgba(*background.rgba)
            ctx.paint()

        ctx.pop_group()
        if background is None:
            ctx.copy_page()
        else:
            ctx.show_page()
    
    def draw_path(self, path, fill, stroke):
        ctx = cairo.Context(self.surface)
        path.as_cairo(ctx)
        
        strokewidth = 1.0  # TODO
        
        if fill.alpha + stroke.alpha == 0.0:
            # Tranverse path here, so point is at correct location
            ctx.traverse()
            return
        
        if fill.alpha and stroke.alpha:
            if stroke.alpha == 1.0:
                # Fast path if no alpha in stroke
                ctx.set_source_rgba(*fill.rgb)
                ctx.fill_preserve()

                ctx.set_source_rgba(*stroke.rgb)
                ctx.set_line_width(strokewidth)
                ctx.stroke()

            else:
                # Draw fill onto intermediate surface so stroke does not overlay fill
                ctx.push_group()

                ctx.set_source_rgba(*fill.rgba)
                ctx.fill_preserve()

                ctx.set_source_rgba(*stroke.rgba)
                ctx.set_operator(cairo.OPERATOR_SOURCE)
                ctx.set_line_width(strokewidth)
                ctx.stroke()

                ctx.pop_group_to_source()
                ctx.paint()
                
        elif fill.alpha:
            ctx.set_source_rgba(*fill.rgba)
            ctx.fill()
        elif stroke.alpha:
            ctx.set_source_rgba(*stroke.rgba)
            ctx.set_line_width(strokewidth)
            ctx.stroke()
        del ctx
    
    def copy_to_surface(self, surface):
        """
        Draw canvas onto to a cairo surface.
        """
        # TODO - this is a cairo specific utility function, to this canvas.
        ctx = cairo.Context(surface)
        ctx.set_source_surface(self.surface)
        ctx.paint()
        del ctx

    def save(self, f, format=None, *args, **kwargs):
        """
        Output current canvas frame to a file or buffer, using cairo to save.
        
        :param f: buffer or filename.
        """
        # TODO - this is handling output formats, saving to buffers etc.
        # 
        if not format:
            if isinstance(f, str):
                suffix = Path(f).suffix
                format = suffix.rsplit('.')[-1] or FORMAT_PNG
            else:
                format = FORMAT_PNG
        
        extents = self.surface.ink_extents()  # TODO: handle ink_extents that don't start at 0, 0
        dimensions = extents[2], extents[3]
        if format == FORMAT_PNG:
            # PNG is special: write_to_png is called after drawing.
            surface = cairo.ImageSurface(cairo.FORMAT_ARGB32, int(dimensions[0]), int(dimensions[1]))
            self.copy_to_surface(surface)
            surface.write_to_png(f)
            return
        elif format == FORMAT_PDF:
            surface = cairo.PDFSurface(f, *dimensions)
        elif format == FORMAT_PS:
            surface = cairo.PDFSurface(f, *dimensions)
        elif format == FORMAT_SVG:
            surface = cairo.SVGSurface(f, *dimensions)
        else:
            raise ValueError(ERROR_UNSUPPORTED_FORMAT)
            
        self.copy_to_surface(surface)
        surface.flush()  # Ensure data written to file or buffer.

    def display(self, format=None, *args, **kwargs):
        """Display to Jupyter cell.
        """
        # TODO - formats supported here may not be the same as for saving.
        from IPython.display import Image, SVG, display as _display

        if format in (FORMAT_PDF, FORMAT_PS):
            raise NotImplementedError(f"{format} is not yet supported.")
        
        if format is None:
            if not args and self.surface.ink_extents() == (.0, .0, .0, .0):
                format = FORMAT_SVG  # 0 size images are legal in SVG unlike many bitmap formats.
            else:
                format = FORMAT_PNG
        
        with BytesIO() as buffer:
            self.save(buffer, format, *args)
            if format == FORMAT_PNG:
                _display(Image(data=buffer.getvalue()))
            elif format == FORMAT_SVG:
                if kwargs.get("syntax_highlight") is True:
                    try:
                        from display_xml import XML
                    except ImportError:
                        raise ImportError('Install display_xml to output as XML:  $ pip3 install display_xml')
                    _display(XML(buffer.getvalue()))
                else:
                    _display(SVG(buffer.getvalue()))
            else:
                # Included for completeness, should have been caught by the call to save().
                raise ValueError(ERROR_UNSUPPORTED_FORMAT)
    
    @contextmanager
    def recording_surface(self, extents=None):
        """
        Context manager to provide a temporary RecordingSurface to draw on.
        
        While context is active, drawing is directed to a RecordingSurface,
        if no exceptions occur, then the content is painted onto the canvases
        original surface.
        
        Buffering drawing commands like this makes it possible to implementing livecoding
        interfaces that don't output graphics when errors occur.
        """
        # TODO - there is probably a less Cairo term for this function than recording_surface.
        _surface = self.surface
        self.surface = surface = cairo.RecordingSurface(cairo.CONTENT_COLOR_ALPHA, extents or self.extents)
        try:
            yield surface
        except:
            raise
        finally:
            self.surface = _surface
        
        # No exceptions were raised, draw
        cr = cairo.Context(_surface)
        cr.set_source_surface(surface)
        cr.paint()


class Variable:
    pass

class IntegerVariable(Variable):
    def __init__(self, default=0, minimum=None, maximum=None):
        self.value = default
        if minimum is None:
            self.minimum = default - 5
        else:
            self.minimum = minimum
        
        if maximum is None:
            self.maximum = default - 5
        else:
            self.maximum = maximum


class Context:
    """
    Define what is visible during the execution of a Shoebot script.
    
    Methods here, delegate drawing operations to the Shoebot Canvas.
    """
    def __init__(self, canvas):
        self._canvas = canvas
        
        self._ns = {}
        
        # default fill / stroke  # TODO - set sensible defaults.
        self.fill(1., 0., 0., 1.)
        self.stroke(None)
        self.background(1., 1., 1.)
        # TODO - if state had better handling, maybe we could do
        # self._state.colors(fill=(...), stroke=(...), background=(...))
        # TODO - look at what PlotDevice does around state.
       
    # Pass "self" to Path instance.
    #     def Path(self):
    #         f"{_Path.__doc__}"
    #         return _Path(self)
    Path = _Path
    
    def size(self, width, height):
        return self._canvas.resize((width, height))

    def translate(self, x, y):
        # TODO - added but without thought on how positioning should work
        self._canvas.translate(x, y)
    
    def background(self, *args):
        """Set background color, or None for to preserve the last frame or page."""
        if args == (None,):
            args = None
        self._background = Color(*(args or ()))
    
    def fill(self, *args):
        """Set fill color, or None for no fill."""
        if args == (None,):
            args = ()
        self._fill = Color(*args)
        
    def stroke(self, *args):
        """Set stroke color, or None for no stroke."""
        if args == (None,):
            args = ()
        self._stroke = Color(*args)
            
    def rect(self, x, y, w, h, draw=True):
        """
        :return: a rectangular Path
        
        By default the Path is added to the canvas, 
        pass draw=False if drawing is not required.
        """
        p = Path(self)
        p.rect(x, y, w, h)
        if draw:
            p.draw()
        return p


def context_as_dict(context):
    """
    :param context: Shoebot Context.
    :return: dict of all public members.
    """
    return {
        name: getattr(context, name)
        for name in dir(context) 
        if not name.startswith('_')
    }
        

class Runner:
    """
    Shoebot "Runner".
    
    Handles the bot lifecycle.
    
    Using aio, extension points are provided allowing user functions to be called before code is executed wait until it runs an iteration.
    """
    def __init__(self, **kwargs):
        self.canvas = Canvas(**kwargs)
        self.context = Context(self.canvas)
        
        self.run_observers = asyncio.Queue()

    async def update_source(self, source):
        """
        Update source code.
        
        Blocks until source is executed by main loop.
        """
        event = SourceChanged(source)
        await self.run_observers.put(event)
        await event.wait()
        return event
                
    def _get_run_observers(self):
        """
        Return list of run observers.
        
        An execution observer is an event that waits on source execution,
        then gets informed when execution completes or fails, so it can
        perform some action.
        """
        observers = []
        try:
            while event := self.run_observers.get_nowait():
                observers.append(event)
        except asyncio.QueueEmpty:
            return observers

    def _get_updated_source(self, observers):
        """
        Return the first observer that is a SourceChanged event,
        this will get to update the source.
        
        Call source_updated on all SourceChanged observers, passing
        in the SourceChanged instance that gets to do the actual
        updating.
        """
        source_changed_event = None
        for event in observers:
            if not isinstance(event, SourceChanged):
                continue
            
            if source_changed_event is None:
                source_changed_event = event
            
            event.source_updated(source_changed_event)
        
        return source_changed_event
    
    async def run_once(self, code, extra_ns=None):
        with self.canvas.recording_surface():
            ns = context_as_dict(self.context)
            ns.update(extra_ns)
            
            with self.canvas.new_page(self.context):
                exec(code, ns, ns)
        
        await asyncio.sleep(0)
    
    async def run_multiple(self, code):
        # TODO - remember what running multiple is in nodeboxes runner.
        await asyncio.sleep(0)
    
    async def run(self, code):
        frame = 1
        while True:
            # TODO - following should be in run, with loop
            observers = self._get_run_observers()
            source_changed = self._get_updated_source(observers)
            if source_changed:
                code = source_changed.source

            try:
                # TODO - handling extra_ns should probably be somewhere else,
                #        (here should be grammar neutral)
                await self.run_once(code, extra_ns={"FRAME": frame, "PAGE_NUM": frame})
            except Exception as e:
                print(traceback.format_exc(), file=sys.stderr)
                if source_changed:
                    source_changed.run_raised_exception(e)
                    self.run_observers.task_done()
                raise
            else:
                # TODO frame handling should be somewhere grammar neutral.
                frame += 1
                for event in observers:
                    event.run_was_successful(code)
                    self.run_observers.task_done()

            ##if frame % 10 == 0:
            ##    clear_output()
            #if source_changed:
            self.canvas.display()  # TODO - just for prototyping.

            await asyncio.sleep(.5)  # TODO let bot set speed

    
    
## Web API
class WebAPI:
    """Web API.

    An HTTP end-point that allows tools such as IDEs to update the source
    being executed.
    """
    def __init__(self, runner):
        self.runner = runner
        
    async def post_source(self, request):
        """
        Post updated source to Squiggle to execute.

        Source is posted in the file field.

        Errors in compilation immediately return with a 500 status code.

        By default, runs the code for one iteration, returning a status 200 on success or 500 if an exception was raised.
        """
        data = await request.post()

        source = data['file']

        if isinstance(source, str):
            unescape_characters = data.get('unescape', "true").lower() in ["true", "yes", "1"]

            # To aid development, you can send source in this field.
            if unescape_characters:
                source_content = source.replace('\\n', '\n')
            else:
                source_content = source
        else:
            filename = source.filename
            source_file = source.file
            source_content = source_file.read()

        try:
            code = compile(source_content, "<string>", "exec")
        except Exception as e:
            return web.json_response({"compiled": {"success": False, "errors": str(e)}},
                                     status=500)
        
        source_changed = await self.runner.update_source(source_content)
        return web.json_response({"compiled": source_changed.success,
                             "ran": source_changed.success,
                             "errors": source_changed.errors or []}, status = 500 if source_changed.errors else 200)
    
    def routes(self):
        return [
            web.post("/source", self.post_source)
        ]

    
async def web_api_runner(runner, host="localhost", port=7780, **kwargs):
    """
    Start the web API.
    
    Accepts new source code POSTed to /source
    
    >>>  await web_api = await web_api_runner(runner)
    """
    api = WebAPI(runner)
    app = web.Application()
    app.add_routes([*api.routes()])

    app_runner = web.AppRunner(app)
    await app_runner.setup()
    site = web.TCPSite(app_runner, host, port)
    await site.start()
    
    return site
    

async def run_webserver(code, runner=None, raise_cancelled=False, **kwargs):
    """
    Convenience function to run bot + webserver.
    
    Users may also take the take the web_api_runner and
    runner to run in their own event loops if needed.
    """
    if runner is None:
        runner = Runner(**kwargs)
    web_api = await web_api_runner(runner, **kwargs)
    try:
        await runner.run(code)
    except (asyncio.CancelledError, Exception) as e:
        await web_api.stop()
        if isinstance(e, asyncio.CancelledError):
            if raise_cancelled:
                raise
        else:
            raise
        
    return runner


## Usage

code = """\
from random import random

size(400, 50)

if FRAME == 4:
    background(None)  # set background to None
fill(random(), random(), random())
p = rect(50*FRAME-50, 0, 45, 45)


"""

# with open("../Squiggle/examples/basic/Squiggle.bot") as f:
#     code = f.read()
#     print(code)

runner = await run_webserver(code, extents=(0., 0., 400., 400.), resizable=True)
print("fin")

In [ ]:
# import inspect
# dir(runner.context)
#runner.context.__dict__.keys()
#
#{name: getattr(runner.context, name) for name in dir(runner.context) if not name.startswith('_')}

In [ ]:
import inspect

dir(runner.context)

#runner.context.__dict__.keys()

{name: getattr(runner.context, name) for name in dir(runner.context) if not name.startswith('_')}

In [ ]:
#help(Path)

In [ ]:
help(Path)